In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt

ticker_symbol = 'LLY'
stock_data = yf.download(ticker_symbol, period='90d', interval='1d')

[*********************100%%**********************]  1 of 1 completed


In [5]:
def get_stock_stats(ticker_symbol, period='90d', interval='1d'):
    stock_data = yf.download(ticker_symbol, period=period, interval=interval)
    daily_returns = stock_data['Adj Close'].pct_change().dropna()
    mu = daily_returns.mean()
    sigma = daily_returns.std()
    return mu, sigma

ticker_symbol = 'LLY'
mu, sigma = get_stock_stats(ticker_symbol)

print("Mean daily return (mu):", mu)
print("Daily standard deviation (sigma):", sigma)


[*********************100%%**********************]  1 of 1 completed

Mean daily return (mu): 0.002787378121377611
Daily standard deviation (sigma): 0.015710157161433397


In [6]:
current_price = stock_data['Adj Close'][-1]
num_simulations = 100000
trading_days_to_expiry = 31

simulated_end_prices = np.zeros(num_simulations)
for i in range(num_simulations):
    random_returns = np.random.normal(mu, sigma, trading_days_to_expiry)
    simulated_end_prices[i] = current_price * np.exp(np.sum(random_returns))

# Equivalent code using list comprehension and vectorized operations
random_returns = np.random.normal(mu, sigma, (num_simulations, trading_days_to_expiry))
simulated_end_prices = current_price * np.exp(np.sum(random_returns, axis=1))

print("Sample of simulated end prices:", simulated_end_prices[:10])

Sample of simulated end prices: [878.17360206 848.03212152 851.45412424 809.50035807 714.99384744
 818.35379333 775.18918505 842.93444757 756.81358165 715.43906863]


In [7]:
strike_price = 830
option_payouts = np.maximum(simulated_end_prices - strike_price, 0, None)

print("Sample of option payouts:", option_payouts[:10])

Sample of option payouts: [48.17360206 18.03212152 21.45412424  0.          0.          0.
  0.         12.93444757  0.          0.        ]


In [8]:
average_payout = np.mean(option_payouts)
risk_free_rate = 5.391 / 100
discount_factor = np.exp(-risk_free_rate * (trading_days_to_expiry / 365))
present_value = average_payout * discount_factor

print("Average payout:", average_payout)
print("Present value of the average payout:", present_value)

Average payout: 16.875924300861847
Present value of the average payout: 16.79883184756653


Based on the Monte Carlo simulation, the average payout for the Eli Lilly call option is estimated at around $16.78. This value is derived from simulating multiple scenarios and calculating the average outcome. However, when we discount this average payout to its present value using the current risk-free rate, it amounts to approximately $16.71.

Comparing this calculated present value to the last traded price of the option, which was $19.50, we notice a slight discrepancy. This difference could indicate several things: the market might be factoring in additional risks or expectations that are not accounted for in our model, or it could be influenced by market sentiment and demand dynamics specific to options on Eli Lilly.

Investors should consider these factors, along with the theoretical value generated by the simulation, when making trading or investment decisions regarding this option.




